Generate latex file to generate summary report

In [ ]:
# imports
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
# Sorting by site name results in the report slides to be organized 
# by the amount of water present in the original scene
SORT_BY_SITENAME=True

In [ ]:
latex_path = Path("../latex")
latex_path.mkdir(exist_ok=True)

In [ ]:
data_path = Path('../data')

df = pd.read_csv(data_path / 'new_validation_table.csv')
df = df[['site_name', 'planet_id', 'dswx_files']]
if SORT_BY_SITENAME:
    df.sort_values('site_name', inplace=True)
df.head()

In [ ]:
df_new_metrics = pd.read_csv('../latex/new_metrics.csv', dtype={'reclassified_strata':str})
df_new_metrics.head()

In [ ]:
df_new_metrics['old_strata'] = df_new_metrics['old_strata'].map(lambda x:int(x))
df_new_metrics['reclassified_strata'] = df_new_metrics['reclassified_strata'].map(lambda x:int(x))

df_new_metrics

In [ ]:
df_new_metrics['Old strata'] = df_new_metrics['old_strata']
df_new_metrics['New strata'] = df_new_metrics['reclassified_strata']

df_new_metrics = df_new_metrics.drop(['old_strata', 'reclassified_strata'], axis=1)

In [ ]:
old_strata_count = df_new_metrics['Old strata'].value_counts()
new_strata_count = df_new_metrics['New strata'].value_counts()

for key in [0, 1, 2, 3]:
    if key not in new_strata_count:
        new_strata_count[key] = 0

    if key not in old_strata_count:
        old_strata_count[key] = 0

old_strata_count = old_strata_count.sort_index()
new_strata_count = new_strata_count.sort_index()

In [ ]:
new_strata_count

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 8))

# PSW
old_keys, old_values = zip(*old_strata_count.items())
new_keys, new_values = zip(*new_strata_count.items())

# Set the width of each bar
bar_width = 0.4

# Calculate the x positions for the bars
old_x = range(0, len(old_keys))
new_x = [x+0.2 for x in old_x]

# Create the bar plots
axs.bar(old_x, old_values, width=bar_width, label='Original validation', alpha=0.8)
axs.bar(new_x, new_values, width=bar_width, label='Expanded validation', alpha=0.8)
axs.set_xlabel('Strata', fontsize=14)
axs.set_ylabel('# of scenes', fontsize=14)
axs.set_xticks([0, 1, 2, 3])
axs.set_xticklabels([0, 1, 2, 3], fontsize=15)
axs.set_yticks([3, 6, 9, 12, 15, 18, 21])
axs.set_yticklabels([3, 6, 9, 12, 15, 18, 21], fontsize=15)
axs.set_title('# of scenes per strata', fontsize=15)
axs.legend(fontsize=14)

plt.savefig('../latex/strata_distribution_plot.png')

In [ ]:
new_strata_count.to_latex("../latex/new_strata_counts_table.tex")
old_strata_count.to_latex("../latex/old_strata_counts_table.tex")

In [ ]:
# Slide template

latex_slide_template = r"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{frame}[fragile]
\frametitle{$<<DSWX_ID>>$}
\begin{center}
\tiny
\begin{verbatim}
DSWx ID: <<DSWX_ID_ORIG>>
PLANET ID: <<PLANET_ID>>
SITE ID: <<SITE_ID>> (NEW STRATA: <<NEW_STRATA>>)

\end{verbatim}
\includegraphics[scale=0.25]{<<DSWX_DIR>>/chip_extents.png}
\vfill
\input{<<DSWX_DIR>>/requirements.tex}
\vfill
\input{<<DSWX_DIR>>/area_summary.tex}
\end{center}
\vfill
\end{frame}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{frame}[fragile]
\frametitle{$<<DSWX_ID>>$}
\begin{center}
\tiny
\begin{verbatim}
DSWx ID: <<DSWX_ID_ORIG>>
PLANET ID: <<PLANET_ID>>
SITE ID: <<SITE_ID>> (NEW STRATA: <<NEW_STRATA>>)
\end{verbatim}
\includegraphics[trim={0 2cm 0 2cm}, scale=0.25, clip]{<<DSWX_DIR>>/classifications.png}
\input{<<DSWX_DIR>>/confusion_matrix.tex}
\end{center}
\end{frame}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""

In [ ]:
# For each Planet side, use template to generate slide
with open(latex_path / "slides.tex", 'w') as f:
    for i, row in df.iterrows():
        
        planet_id = row.planet_id
        dswx_id_orig = row.dswx_files.split('/')[-1][:-12]
        dswx_id = dswx_id_orig.replace('_', '\_').replace('-', '{\\text -}')

        tmp_text = latex_slide_template.replace("<<DSWX_ID>>", str(dswx_id))
        tmp_text = tmp_text.replace("<<DSWX_ID_ORIG>>", str(dswx_id_orig))
        tmp_text = tmp_text.replace("<<DSWX_DIR>>", str(planet_id))
        tmp_text = tmp_text.replace("<<<Not Passed>>>", "\textcolor{red}{Not Passed}")
        tmp_text = tmp_text.replace("<<PLANET_ID>>", planet_id)
        tmp_text = tmp_text.replace("<<SITE_ID>>", row.site_name)

        _row = df_new_metrics[df_new_metrics.planet_id == planet_id]

        tmp_text = tmp_text.replace("<<NEW_STRATA>>", str(int(_row['New strata'].values[0])))

        f.write(tmp_text)

### Create a summary table for DSWx performance

In [ ]:
temp_df = df_new_metrics

# trim, in case there are more than 52 records
temp_df = temp_df.iloc[0:52]

n_scenes = len(temp_df)

old_psw_dswx = np.round(100*np.sum(temp_df['Old_PSW_mean_accuracy'] > 70) / n_scenes, 2)
old_osw_dswx = np.round(100*np.sum(temp_df['Old_OSW_mean_accuracy'] > 80) / n_scenes, 2)

new_psw_dswx = np.round(100*np.sum(temp_df['New_PSW_mean_accuracy'] > 70) / n_scenes, 2)
new_osw_dswx = np.round(100*np.sum(temp_df['New_OSW_mean_accuracy'] > 80) / n_scenes, 2)

print(f"% scenes passing PSW with old validation {old_psw_dswx}\%")
print(f"% scenes passing OSW with old validation {old_osw_dswx}\%")

print(f"% scenes passing PSW with new validation {new_psw_dswx}\%")
print(f"% scenes passing OSW with new validation {new_osw_dswx}\%")

formatted_not_pass = "\\textcolor{red}{Not Passed}"
formatted_pass = "\\textcolor{green}{Passed}"

# old validation pass/fail:
old_psw_str = f"\\textcolor{{red}}{{{old_psw_dswx}\%}} ({formatted_not_pass})" if old_psw_dswx < 70 else f"\\textcolor{{green}}{{{old_psw_dswx}\%}} ({formatted_pass})"
old_osw_str = f"\\textcolor{{red}}{old_osw_dswx}\%}} ({formatted_not_pass})" if old_osw_dswx < 80 else f"\\textcolor{{green}}{{{old_osw_dswx}\%}} ({formatted_pass})"

# new validation pass/fail:
new_psw_str = f"\\textcolor{{red}}{{{new_psw_dswx}\%}} ({formatted_not_pass})" if new_psw_dswx < 70 else f"\\textcolor{{green}}{{{new_psw_dswx}\%}} ({formatted_pass})"
new_osw_str = f"\\textcolor{{red}}{{{new_osw_dswx}\%}} ({formatted_not_pass})" if new_osw_dswx < 80 else f"\\textcolor{{green}}{{{new_osw_dswx}\%}} ({formatted_pass})"

summary_table = {"Class":["PSW", "OSW"], 
"Original Validation OPERA Req.":[old_psw_str, old_osw_str], 
"Expanded Validation OPERA Req.":[new_psw_str, new_osw_str]}

summary_table_df = pd.DataFrame(summary_table)
summary_table_df
summary_table_df.to_latex("../latex/summary.tex")

In [ ]:
old_osw_passing = dict({'0':0, '1':0, '2':0, '3':0})
old_psw_passing = dict({'0':0, '1':0, '2':0, '3':0})
new_osw_passing = dict({'0':0, '1':0, '2':0, '3':0})
new_psw_passing = dict({'0':0, '1':0, '2':0, '3':0})

for _, row in temp_df.iterrows():
    strata = str(row['Old strata'])
    new_strata = str(row['New strata'])

    o_psw_acc = row['Old_PSW_mean_accuracy']
    o_osw_acc = row['Old_OSW_mean_accuracy']
    
    n_psw_acc = row['New_PSW_mean_accuracy']
    n_osw_acc = row['New_OSW_mean_accuracy']

    if o_psw_acc >= 70:
        old_psw_passing[strata] += 1
    if o_osw_acc >= 80:
        old_osw_passing[strata] += 1

    if n_psw_acc >= 70:
        new_psw_passing[new_strata] += 1
    if n_osw_acc >= 80:
        new_osw_passing[new_strata] += 1

print(old_psw_passing, old_osw_passing, new_psw_passing, new_osw_passing)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# PSW
old_keys, old_values = zip(*old_psw_passing.items())
new_keys, new_values = zip(*new_psw_passing.items())

# Set the width of each bar
bar_width = 0.4

# Calculate the x positions for the bars
old_x = range(0, len(old_keys))
new_x = [x+0.2 for x in old_x]

# Create the bar plots
axs[0].bar(old_x, old_values, width=bar_width, label='Original validation', alpha=0.8)
axs[0].bar(new_x, new_values, width=bar_width, label='Expanded validation', alpha=0.8)
axs[0].set_xlabel('Strata', fontsize=14)
axs[0].set_ylabel('Passing cases', fontsize=14)
axs[0].set_xticks([0, 1, 2, 3])
axs[0].set_xticklabels([0, 1, 2, 3], fontsize=15)
axs[0].set_yticks([3, 6, 9, 12, 15, 18, 21, 24])
axs[0].set_yticklabels([3, 6, 9, 12, 15, 18, 21, 24], fontsize=15)
axs[0].set_title('# of passing PSW', fontsize=15)
axs[0].legend(fontsize=14)


# OSW
old_keys, old_values = zip(*old_osw_passing.items())
new_keys, new_values = zip(*new_osw_passing.items())

# Set the width of each bar
bar_width = 0.4

# Calculate the x positions for the bars
old_x = range(0, len(old_keys))
new_x = [x+0.2 for x in old_x]

# Create the bar plots
axs[1].bar(old_x, old_values, width=bar_width, label='Original validation', alpha=0.8)
axs[1].bar(new_x, new_values, width=bar_width, label='Expanded validation', alpha=0.8)
axs[1].set_xlabel('Strata', fontsize=14)
axs[1].set_xticks([0, 1, 2, 3])
axs[1].set_xticklabels([0, 1, 2, 3], fontsize=15)
axs[1].set_yticks([3, 6, 9, 12, 15, 18, 21, 24])
axs[1].set_yticklabels([3, 6, 9, 12, 15, 18, 21, 24], fontsize=15)
axs[1].set_title('# of passing OSW', fontsize=15)
axs[1].legend(fontsize=14)

plt.savefig('../latex/performance_bar_plot.png')

In [ ]:
original_osw_passing = dict({'0':0, '1':0, '2':0, '3':0})
original_psw_passing = dict({'0':0, '1':0, '2':0, '3':0})

for _, row in temp_df.iterrows():
    strata = str(row['Old strata'])

    n_psw_acc = row['Old_PSW_mean_accuracy']
    n_osw_acc = row['Old_OSW_mean_accuracy']

    if n_psw_acc >= 70:
        original_psw_passing[strata] += 1
    if n_osw_acc >= 80:
        original_osw_passing[strata] += 1

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 8))

# PSW
psw_keys, psw_values = zip(*original_psw_passing.items())
osw_keys, osw_values = zip(*original_osw_passing.items())
_, old_strata_counts = zip(*old_strata_count.items())

# Set the width of each bar
bar_width = 0.4

# Calculate the x positions for the bars
old_x = range(0, len(psw_keys))
new_x = [x+0.2 for x in old_x]
newer_x = [x+0.1 for x in old_x]

# Create the bar plots
axs.bar(old_x, psw_values, width=bar_width, label='PSW passing', alpha=0.8)
axs.bar(new_x, osw_values, width=bar_width, label='OSW passing', alpha=0.8)
axs.bar(newer_x, old_strata_counts, width=bar_width+0.2, fill=False, label='Total # of scenes in strata')
axs.set_xlabel('Strata', fontsize=14)
axs.set_ylabel('Count', fontsize=14)
axs.set_xticks([0, 1, 2, 3])
axs.set_xticklabels([0, 1, 2, 3], fontsize=15)
axs.set_yticks([3, 6, 9, 12, 15, 18, 21])
axs.set_yticklabels([3, 6, 9, 12, 15, 18, 21], fontsize=15)
axs.set_title('Passing PSW/OSW in original validation', fontsize=15)
axs.legend(fontsize=14, loc='upper left')


plt.savefig('../latex/pass_fail_original_distribution.png')

In [ ]:
current_osw_passing = dict({'0':0, '1':0, '2':0, '3':0})
current_psw_passing = dict({'0':0, '1':0, '2':0, '3':0})

for _, row in temp_df.iterrows():
    strata = str(row['New strata'])
    # print(strata)

    n_psw_acc = row['New_PSW_mean_accuracy']
    n_osw_acc = row['New_OSW_mean_accuracy']

    if n_psw_acc >= 70:
        current_psw_passing[strata] += 1
    if n_osw_acc >= 80:
        current_osw_passing[strata] += 1

In [ ]:
temp_df.head()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12, 8))

# PSW
psw_keys, psw_values = zip(*current_psw_passing.items())
osw_keys, osw_values = zip(*current_osw_passing.items())
_, new_strata_counts = zip(*new_strata_count.items())

# Set the width of each bar
bar_width = 0.4

# Calculate the x positions for the bars
old_x = range(0, len(psw_keys))
new_x = [x+0.2 for x in old_x]
newer_x = [x+0.1 for x in old_x]

# Create the bar plots
axs.bar(old_x, psw_values, width=bar_width, label='PSW passing', alpha=0.8)
axs.bar(new_x, osw_values, width=bar_width, label='OSW passing', alpha=0.8)
axs.bar(newer_x, new_strata_counts, width=bar_width+0.2, fill=False, label='Total # of scenes in strata')
axs.set_xlabel('Strata', fontsize=14)
axs.set_ylabel('Count', fontsize=14)
axs.set_xticks([0, 1, 2, 3])
axs.set_xticklabels([0, 1, 2, 3], fontsize=15)
axs.set_yticks([3, 6, 9, 12, 15, 18, 21])
axs.set_yticklabels([3, 6, 9, 12, 15, 18, 21], fontsize=15)
axs.set_title('Passing PSW/OSW in expanded validation', fontsize=15)
axs.legend(fontsize=14, loc='upper left')


plt.savefig('../latex/pass_fail_new_distribution.png')